##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 序贯模型

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/guide/keras/sequential_model"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/keras/sequential_model.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行 </a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/keras/sequential_model.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/guide/keras/sequential_model.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>
</td>
</table>

## 设置

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## 何时使用序贯模型

`Sequential` 模型适用于**简单的层堆叠**，其中每一层都**恰好具有一个输入张量和一个输出张量**。

从原理上看，以下 `Sequential` 模型：

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

等效于以下函数：

In [ ]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

序贯模型**不适用于**以下情况：

- 您的模型有多个输入或多个输出
- 您的任何一层都有多个输入或多个输出
- 您需要进行层共享
- 您需要非线性拓扑（例如，残差连接、多分支模型）

## 创建一个序贯模型

可以通过将层列表传递给 Sequential 构造函数来创建序贯模型：

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

可以通过 `layers` 特性访问模型的层：

In [ ]:
model.layers

还可以通过 `add()` 方法增量式创建一个序贯模型：

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

请注意，还有一个相应的 `pop()` 方法可以移除层：序贯模型的行为与层列表非常相似。

In [ ]:
model.pop()
print(len(model.layers))  # 2

另请注意，Sequential 构造函数接受 `name` 参数，就像 Keras 中的任何层或模型一样。这对于使用在语义上有意义的名称注解 TensorBoard 计算图而言十分有用。

In [ ]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

## 预先指定输入形状

通常，Keras 中的所有层都需要了解其输入的形状，以便能够创建其权重。因此，当您创建如下所示的层时，它最初没有权重：

In [ ]:
layer = layers.Dense(3)
layer.weights  # Empty

由于权重的形状取决于输入的形状，因此在输入上首次调用该层时，会创建层的权重：

In [ ]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

自然，这也适用于序贯模型。当您实例化没有输入形状的序贯模型时，不会“构建”模型：它没有权重（调用 `model.weights` 会导致描述这种情况的错误）。当模型首次看到一些输入数据时，会创建权重：

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

一旦“构建”模型，就可以调用它的 `summary()` 方法来显示模型内容：

In [ ]:
model.summary()

但是，当增量式构建序贯模型时，能够显示到目前为止的模型摘要（包括当前输出形状）十分有用。在这种情况下，应当通过将 `Input` 对象传递给模型来启动您的模型，这样，模型可以从一开始就了解其输入形状：

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

请注意，`Input` 对象不会显示为 `model.layers` 的一部分，因为它不是一个层：

In [ ]:
model.layers

一种简单的替代方法是将 `input_shape` 参数传递给您的第一个层：

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

使用此类预定义输入形状构建的模型始终具有权重（甚至在看到任何数据之前），并且始终具有定义的输出形状。

一般而言，推荐的最佳做法是始终事先指定序贯模型的输入形状（如果您知道）。

## 常见的调试工作流：`add()` + `summary()`

在构建新的 Sequential 架构时，使用 `add()` 增量式堆叠层并经常打印模型摘要十分有用。例如，这让您可以监视 `Conv2D` 和 `MaxPooling2D` 层的堆叠如何对图像特征映射进行下采样：

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

非常实用，对吗？


## 拥有模型后的操作

一旦模型架构准备就绪，您可以继续执行以下操作：

- 训练您的模型，评估模型并运行推断。请参阅我们的[使用内置循环进行训练和评估指南](https://tensorflow.google.cn/guide/keras/train_and_evaluate/)。
- 将模型保存到磁盘并恢复模型。请参阅我们的[序列化和保存指南](https://tensorflow.google.cn/guide/keras/save_and_serialize/)。
- 通过利用多个 GPU 来加快模型训练速度。请参阅我们的[多 GPU 和分布式训练指南](https://keras.io/guides/distributed_training/)。

## 使用序贯模型提取特征

序贯模型一旦构建完成，它的行为就类似于[函数式 API 模型](https://tensorflow.google.cn/guide/keras/functional/)。这意味着每个层都具有 `input` 和 `output` 特性。这些特性可用于执行各种操作，例如快速创建一个模型来提取序贯模型中所有中间层的输出：

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

下面是一个仅从一个层中提取特征的类似示例：

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

## 使用序贯模型进行迁移学习

迁移学习包括冻结模型中的底层以及仅训练顶层。如果您不熟悉，请务必阅读我们的[迁移学习指南](https://tensorflow.google.cn/guide/keras/transfer_learning/)。

下面是涉及序贯模型的两个常见迁移学习蓝图。

首先，假设您有一个序贯模型，并且想要冻结除最后一层之外的所有层。在这种情况下，您只需遍历 `model.layers` 并在除最后一层之外的每个层上设置 `layer.trainable = False`。如下所示：

```python
model = keras.Sequential([
    keras.Input(shape=(784))
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# Presumably you would want to first load pre-trained weights.
model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
model.compile(...)
model.fit(...)
```

另一个常见蓝图是使用序贯模型来堆叠预训练模型和一些新初始化的分类层。如下所示：

```python
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

# Compile & train
model.compile(...)
model.fit(...)
```

如果您进行迁移学习，可能会经常使用这两种模式。

关于序贯模型，这就是您需要了解的全部内容！

要详细了解如何在 Keras 中构建模型，请参阅：

- [函数式 API 指南](https://tensorflow.google.cn/guide/keras/functional/)
- [通过子类化创建新的层和模型指南](https://tensorflow.google.cn/guide/keras/custom_layers_and_models/)